In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import math
import gc

NotFoundError: /home/DAVIDSON/ilnavani/.venvs/TensorFlow-GPU/lib/python3.8/site-packages/tensorflow/core/kernels/libtfkernel_sobol_op.so: undefined symbol: _ZN10tensorflow15OpKernelContext21CtxFailureWithWarningEPKciRKN3tsl6StatusE

In [134]:
df = pd.read_csv("/home/DAVIDSON/brwiedenbeck/public/NLP/quora.csv")

In [135]:
train_data, test_data = train_test_split(df, train_size=0.80, stratify = df['target'], random_state=42)

In [136]:
train_data, val_data = train_test_split(train_data, train_size=0.80, stratify = train_data['target'], random_state=42)

In [5]:
print(train_data.shape)
print(train_data['target'].value_counts())
print(val_data.shape)
print(val_data['target'].value_counts())
print(test_data.shape)
print(test_data['target'].value_counts())

(835917, 3)
0    784199
1     51718
Name: target, dtype: int64
(208980, 3)
0    196050
1     12930
Name: target, dtype: int64


In [5]:
# Extract relevant text and labels 
text = df['question_text']
labels = df['target']

In [6]:
# Split train and test data (80-20 split)
train_data, test_data = train_test_split(text, train_size=0.80, random_state=42)
train_labels, test_labels = train_test_split(labels, train_size=0.80, random_state=42)

In [7]:
# Extract validation set (20%) from training data
train_data, val_data = train_test_split(train_data, train_size=0.80, random_state=42)
train_labels, val_labels = train_test_split(train_labels, train_size=0.80, random_state=42)

In [8]:
np.save("validation_labels", val_labels)

In [ ]:
# Create a debugging set from train data (1% of data)
#train_data, debug_data = train_test_split(train_data, train_size=0.99, random_state=42)
#train_labels, debug_labels = train_test_split(train_labels, train_size=0.99, random_state=42)
#1630 gives batches of 512

### Preprocessing Training set

In [6]:
# Split training data into 10 batches
train_data = np.array_split(train_data, 12)

In [7]:
len(train_data[0])

69660

In [10]:
train_labels = np.array_split(train_labels, 12)

In [11]:
len(train_labels[0])

69660

In [23]:
np.save("train_labels_0", train_labels[0])

In [137]:
# Get glove embeddings
# "embedding" is a dictionary of raw embeddings of words
f = open("glove.6B.100d.txt")
lines = f.readlines()
embedding = dict()
for line in lines:
        parts = line.split()
        embedding[parts[0]] = np.asarray(parts[1:], dtype='float32')
        #, dtype='float32'

In [42]:
def glove_embedding(train_data):

    tokens = []
    for elem in train_data:
        token = text_to_word_sequence(elem)
        tokens.append(token) 

    train_embedding = []
    for question in tokens:
        matrix = np.zeros((len(question), 100))
        for index, word in enumerate(question):
            vector_rep = embedding.get(word)
            if vector_rep is not None:
                matrix[index] = vector_rep
            train_embedding.append(matrix)
    #train_embedding = np.array(train_embedding, dtype=object)
    train_embedding = pad_sequences(train_embedding, padding = 'post', dtype='float64')

#tokens.append(batch_tokens)
#tokens = pd.Series(tokens)

In [138]:
def glove_embedding(train_data):
    
    tokens = text_to_word_sequence(train_data)
    matrix = np.zeros((len(tokens), 100))
    for index, word in enumerate(tokens):
        vector_rep = embedding.get(word)
        if vector_rep is not None:
            matrix[index] = vector_rep
    matrix = np.array(matrix, dtype=object)
    return matrix

In [221]:
embedding1 = glove_embedding(train_data.iloc[0,1])

In [222]:
print(type(embedding1))

<class 'numpy.ndarray'>


In [4]:
np.save("train_data_0", train_data)

In [ ]:
#labels 
train_embedding = []
for question in tokens:
    matrix = np.zeros((len(question), 100))
    for index, word in enumerate(question):
        vector_rep = embedding.get(word)
        if vector_rep is not None:
            matrix[index] = vector_rep
    train_embedding.append(matrix)


In [ ]:
# Create list of embeddings for each question in the training set
train_embedding = []
for batch in tokens:
    batch_embedding = []
    for question in batch:
        matrix = np.zeros((len(question), 100))
        for index, word in enumerate(question):
            vector_rep = embedding.get(word)
            if vector_rep is not None:
                matrix[index] = vector_rep
        batch_embedding.append(matrix)
    batch_embedding = pad_sequences(batch_embedding)
    train_embedding.append(batch_embedding)
    
# Padding to make all sequences the same length
#train_embedding = pad_sequences(train_embedding)

### Preprocessing Debug set

In [ ]:
# Convert questions to word tokens
debug_tokens = []

from tensorflow.keras.preprocessing.text import text_to_word_sequence
for elem in debug_data:
    tokens = text_to_word_sequence(elem)
    debug_tokens.append(tokens)

debug_tokens = pd.Series(debug_tokens)

In [ ]:
# Convert each question to numeric tokens  -- not using this right now
tokenizer = Tokenizer()
tokenizer.fit_on_texts(debug_data)
debug_sequences = tokenizer.texts_to_sequences(debug_data)

vocab_size = len(tokenizer.word_index)+1

In [12]:
# Get glove embeddings
# Embedding is a dictionary of raw embeddings of words
f = open("glove.6B.100d.txt")
lines = f.readlines()
embedding = dict()
for line in lines:
        parts = line.split()
        embedding[parts[0]] = np.asarray(parts[1:], dtype='float32')

In [ ]:
# Create list of embeddings for each question in the debug set
debug_embedding = []
for question in debug_tokens:
    matrix = np.zeros((len(question), 100))
    for index, word in enumerate(question):
        vector_rep = embedding.get(word)
        if vector_rep is not None:
            matrix[index] = vector_rep
    debug_embedding.append(matrix)
    
# Padding to make all sequences the same length
debug_embedding = pad_sequences(debug_embedding, dtype="long", truncating="post", padding="post")

### Preprocessing Validation Data

In [9]:
# Convert each validation question to word tokens
val_tokens = []

from tensorflow.keras.preprocessing.text import text_to_word_sequence
for elem in val_data:
    tokens = text_to_word_sequence(elem)
    val_tokens.append(tokens)

val_tokens = pd.Series(val_tokens)

In [10]:
len(val_tokens)

208980

In [10]:
# Create list of embeddings for each question in the validation set
val_embedding = []
for question in val_tokens:
    matrix = np.zeros((len(question), 100))
    for index, word in enumerate(question):
        vector_rep = embedding.get(word)
        if vector_rep is not None:
            matrix[index] = vector_rep
    val_embedding.append(matrix)

In [11]:
val_embedding = np.array(val_embedding, dtype=object)

In [ ]:
val_embedding = pad_sequences(val_embedding, padding = 'post', dtype='float64')

In [ ]:
# Give embedded questions as input to model. You don't need the embedding layer anymore.
# Embedding layer is: #model.add(tf.keras.layers.Embedding(vocab_size, 300, weights = [debug_embedding], input_length=300, trainable = False))
# No need to pad sequences for LSTM bec it takes in one word at a time. You may need to pad for transformers?
# Set 'return_sequences = True' for an LSTM that is feeding into another LSTM
# Very high accuracies in the 95% range are plausible 
# For this dataset there are a lot more sincere questions than insincere
# So create a confusion matrix to see if we are in fact making good predictions on the insincere questions
# Loop over the epochs and loop over the files
# regularizers and dropout

### Loading Data

In [2]:
train_data = np.load(file = "train_data_0.npy", allow_pickle = True)

In [14]:
val_embedding = np.load(file = "validation_data.npy", allow_pickle = True)

In [15]:
len(val_embedding)

208980

In [21]:
len(val_embedding[1])

16

In [3]:
val_embedding = np.array_split(val_embedding, 2)

In [23]:
for elem in range(208980):
    val_embedding[elem] = pad_sequences(val_embedding[elem], padding = 'post', dtype='float64', maxlen = 63)

In [33]:
val_embedding[0] = pad_sequences(val_embedding[0], maxlen = 63, padding = 'post', dtype='float64')

In [ ]:
val_size = 10000
val_embedding = []
for question in val_data["question_text"][:val_size]:
    val_embedding.append(glove_embedding(question))
    
val_embedding = np.array(val_embedding, dtype=object)
val_embedding = pad_sequences(val_embedding, padding = 'post', dtype = 'float32', maxlen = 100)
val_labels = np.array(val_data["target"][:val_size], dtype='int32')

validation_data=(val_embedding, val_labels)

### Training

Outer loop goes through epochs. Inner loop goes through files. 
Within each inner loop, load each file (generate string name by changing the index) and fit it for 1 epoch (so you only have one file in memory at a time) 
Afterwards run through validation data

In [254]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.LSTM(units = 128))
model.add(tf.keras.layers.Dense(16, activation = 'sigmoid'))
model.add(tf.keras.layers.Dropout(0.2))
#model.add(tf.keras.layers.Dense(2, activation = 'softmax'))
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

In [290]:
model = tf.keras.Sequential()

model.add((tf.keras.layers.LSTM(128,return_sequences=True)))
model.add(tf.keras.layers.GlobalMaxPool1D())
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))


In [193]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy

In [291]:
model.compile(optimizer=Adam(0.001), loss="binary_crossentropy", metrics=['accuracy'])

In [194]:
model.compile(optimizer=Adam(), 
        loss='categorical_crossentropy', 
        metrics=[Accuracy()])

In [209]:
train_data["target"] = keras.utils.to_categorical(train_data["target"], 2)

In [210]:
val_data["target"] = keras.utils.to_categorical(val_data["target"], 2)

In [4]:
#callback = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3, restore_best_weights = True)]

In [230]:
# Generator when reading one file at a time 
batch_size = 256

# create an generator for the training set
def batch_gen(train_df, train_labels):
    # number of batches
    n_batches = math.ceil(len(train_df) / batch_size)
    while True: 
        #train_df = train_df.sample(frac=1.)  # Shuffle the data.
        for i in range(n_batches):
            texts = train_df[i*batch_size:(i+1)*batch_size] 
            yield texts, train_labels[i*batch_size:(i+1)*batch_size] # generator: embedding and target

In [283]:
# Generator for entire dataset 
batch_size = 256

def train_gen(train_data):
    num_batches = math.ceil(len(train_data) / batch_size)
    while True: 
        train_data = train_data.sample(frac=1.) #shuffle the data
        for i in range(num_batches):
            train_questions = []
            batch = train_data.iloc[i * batch_size: (i + 1) * batch_size, 1]
            for question in batch:
                train_questions.append(glove_embedding(question))
            train_questions = np.array(train_questions, dtype = object)
            #text_arr.append([glove_embedding(text) for text in texts], dtype=object)
            #text_arr = np.array([glove_embedding(text) for text in texts], dtype=object)
            train_questions = pad_sequences(train_questions, padding = 'post', dtype = 'float32', maxlen = 100)
            #text_arr = tf.cast(text_arr, dtype=tf.int32)
            #labels_arr = np.array(train_data["target"][i * batch_size:(i + 1) * batch_size])
            #labels_arr = tf.cast(text_arr, dtype=tf.int32)
            yield train_questions, np.array(train_data["target"][i * batch_size:(i + 1) * batch_size])

In [ ]:
# Generator for validation data
batch_size = 256

def val_gen(val_data):
    num_batches = math.ceil(len(val_data) / batch_size)
    while True: 
        val_data = val_data.sample(frac=1.)
        for i in range(num_batches):
            val_questions = []
            batch = val_data.iloc[i * batch_size: (i + 1) * batch_size, 1]
            for question in batch:
                val_questions.append(glove_embedding(question))
        val_questions = np.array(val_questions, dtype = object)
        val_questions = pad_sequences(val_questions, padding = 'post', dtype = 'float32', maxlen = 100)
        yield val_questions, np.array(val_data["target"][i * batch_size:(i + 1) * batch_size])

In [295]:
# mg is a combination of word embedding and target value. 
train_generator = train_gen(train_data)
val_generator = val_gen(val_data)

In [280]:
# training on mg, tuples of embedding and target
history = model.fit(train_generator, epochs=5, steps_per_epoch=1000, verbose = 1)

Epoch 1/5
1000/1000 [==============================] - 245s 241ms/step - loss: 0.1427 - accuracy: 0.9475
Epoch 2/5
1000/1000 [==============================] - 265s 265ms/step - loss: 0.1201 - accuracy: 0.9539
Epoch 3/5
1000/1000 [==============================] - 268s 268ms/step - loss: 0.1161 - accuracy: 0.9548
Epoch 4/5
1000/1000 [==============================] - 264s 264ms/step - loss: 0.1112 - accuracy: 0.9563
Epoch 5/5
1000/1000 [==============================] - 263s 263ms/step - loss: 0.1097 - accuracy: 0.9570


In [293]:
model = tf.keras.Sequential()

model.add((tf.keras.layers.LSTM(64,return_sequences=True)))
model.add(tf.keras.layers.GlobalMaxPool1D())
model.add(tf.keras.layers.Dense(32,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [294]:
model.compile(optimizer=Adam(0.001), loss="binary_crossentropy", metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, epochs=10, steps_per_epoch = 1000, validation_data = val_generator, validation_steps = 500, verbose = 1)


In [ ]:
for epoch in range(10):
    for i in range(12): # 12 files of training data
        data_filename = "train_data_"+str(i)+".npy"
        train_data = np.load(file = data_filename, allow_pickle = True)
        
        labels_filename = "train_labels_"+str(i)+".npy"
        train_labels = np.load(file = labels_filename, allow_pickle = True)
        
        model.fit(train_data, train_labels, batch_size = 256, epochs = 1)
        
        # hosuekeeping
        del train_data
        del train_labels
        gc.collect()
        tf.keras.backend.clear_session()

273/273 [==============================] - 26s 91ms/step - loss: 0.2424 - accuracy: 0.9363


### Confusion Matrix

In [ ]:
import itertools
from sklearn.metrics import confusion_matrix

def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15): 
    
  # Create the confustion matrix
    cm = confusion_matrix(y_true, y_pred)
    cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
    n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure 
    fig, ax = plt.subplots(figsize=figsize)
    cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors represent how 'correct' a class is, darker == better
    fig.colorbar(cax)

  # Check for list of classes
    if classes:
        labels = classes
    else:
        labels = np.arange(cm.shape[0])
  
  # Label the axes
    ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes), 
         xticklabels=labels, # axes will labeled with ints
         yticklabels=labels)
  
  # Make x-axis labels appear on bottom
    ax.xaxis.set_label_position("bottom")
    ax.xaxis.tick_bottom()

  # Set the threshold for different colors
    threshold = (cm.max() + cm.min()) / 2.

In [ ]:
predictions_val = model.predict(x=val_embedding, verbose = 0)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(val_label, rounded_predictions_val)

In [ ]:
matrix = confusion_matrix(y_true=val_label, y_pred=rounded_predictions_val)

In [ ]:
# Get the percentage of correct predictions per class
averages = matrix.diagonal()/matrix.sum(axis=1)
print("Array containing percentage of true predictions by class: \n", averages)

# Get average percentage of correct predictions across classes 
sum = 0
for average in averages:
    sum += average
total_average = sum/2
print("Average percentage of true predictions: ", total_average)

In [ ]:
train_data_0 = np.load(file = "train_data_0.npy")
train_data_1 = np.load(file = "train_data_1.npy")
train_data_2 = np.load(file = "train_data_2.npy")
train_data_3 = np.load(file = "train_data_3.npy")
train_data_4 = np.load(file = "train_data_4.npy")
train_data_5 = np.load(file = "train_data_5.npy")
train_data_6 = np.load(file = "train_data_6.npy")
train_data_7 = np.load(file = "train_data_7.npy")
train_data_8 = np.load(file = "train_data_8.npy")
train_data_9 = np.load(file = "train_data_9.npy")

In [19]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(16,(4,4),strides=(2,2),input_shape = (80, 60, 32), padding = "valid"))

In [17]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(20,(7,7),strides=(3,3),input_shape = (300,200,3), padding = "valid"))

In [20]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 39, 29, 16)        8208      
Total params: 8,208
Trainable params: 8,208
Non-trainable params: 0
_________________________________________________________________
